# Background Replacement
## Bikes

In [ ]:
import os
if os.getcwd().split('\\')[-1] != 'InsertDiffusion' and os.getcwd().split('/')[-1] != 'InsertDiffusion':
    os.chdir('..')

import utils
import importlib
importlib.reload(utils)
%load_ext autoreload
%autoreload 2

In [ ]:
COLORIZATION_MODEL = 'diffusers/stable-diffusion-xl-1.0-inpainting-0.1'
UPSCALING_MODEL = 'stabilityai/stable-diffusion-x4-upscaler'
COLORIZATION_NEGATIVE_PROMPT = "black and white, black frame, silhouette, motorbike, toy, clay, model, missing saddle, high saddle, details, detailed, greyscale, duplicate, multiple, detached, shadow, contact shadow, drop shadow, reflection, ground, unrealistic, bad, distorted, ugly, weird"
COLORIZATION_FILL_HOLES = True
COLORIZATION_DILATION = 8
COLORIZATION_STRENGTH = 0.91
COLORIZATION_GUIDANCE = 17
COLORIZATION_MASK_THRESHOLD = 170
COLORIZATION_STEPS = 30

INPAINTING_MASK_THRESHOLD = 230
INPAINTING_MODEL = 'stabilityai/stable-diffusion-2-inpainting'
INPAINTING_NEGATIVE_PROMPT = "wrong proportions, toy, black frame, motorbike, silhouette, model, clay, high saddle, large wheels, text, above floor, flying, changed bike color, white background, duplicate, multiple, people, basket, distortion, low quality, worst, ugly, fuzzy, blurry, cartoon, simple, art"
INPAINTING_GUIDANCE = 15
INPAINTING_STEPS = 75

IMG2IMG_MODEL = 'stabilityai/stable-diffusion-xl-refiner-1.0'
IMG2IMG_STRENGTH = 0.2
IMG2IMG_GUIDANCE = 7.5

In [ ]:
from PIL import Image
bikes = [Image.open('./test_images/for_eval/bikes/sketch/' + x) for x in sorted(os.listdir('./test_images/for_eval/bikes/sketch/'))]

with open('./test_images/for_eval/bikes/prompts.txt', 'r') as f:
    prompts = f.readlines()
bike_indices = [int(x.split('--')[0]) for x in prompts]
colorization_prompts = [x.split('--')[1] for x in prompts]
inpainting_prompts = [x.split('--')[2] for x in prompts]

In [ ]:
def colorization(image: Image, colorization_model: str, upscaling_model: str, colorization_prompt: str, colorization_negative_prompt: str, fill_holes: bool, dilation: int, strength: float, prompt_guidance: float):
    colorized = utils.sd_colorization(colorization_model, upscaling_model, image, colorization_prompt, negative_prompt=colorization_negative_prompt, fill_holes=fill_holes, dilation_iterations=dilation, colorization_strength=strength, prompt_guidance=prompt_guidance)
    return colorized

def insert_diffusion(image: Image, mask_threshold: int, prompt: str, negative_prompt: str, img2img_model: str, inpainting_model: str, img2img_strength: float, inpainting_steps: int, inpainting_guidance: float, img2img_guidance: float) -> Image:
    mask = utils.get_mask_from_image(image, mask_threshold)
    inpainted = utils.sd_inpainting(inpainting_model, image, mask, prompt, negative_prompt, inpainting_guidance, inpainting_steps)
    result = utils.sd_img2img(img2img_model, inpainted, prompt, negative_prompt, img2img_strength, img2img_guidance)
    return result

In [ ]:
os.makedirs('./test_images/eval_results/ours_ablation/bikes/', exist_ok=True)

for i, bike in enumerate(bikes):
    colorized = colorization(bike, COLORIZATION_MODEL, UPSCALING_MODEL, colorization_prompts[i], COLORIZATION_NEGATIVE_PROMPT, True, COLORIZATION_DILATION, COLORIZATION_STRENGTH, COLORIZATION_GUIDANCE)
    inserted = insert_diffusion(colorized, INPAINTING_MASK_THRESHOLD, inpainting_prompts[i], INPAINTING_NEGATIVE_PROMPT, IMG2IMG_MODEL, INPAINTING_MODEL, IMG2IMG_STRENGTH, INPAINTING_STEPS, INPAINTING_GUIDANCE, IMG2IMG_GUIDANCE)
    print(inpainting_prompts[i])
    display(inserted)
    inserted.save('./test_images/eval_results/ours_ablation/bikes/' + inpainting_prompts[i].split(',')[0] + '.png')

## Products

In [ ]:
import os
if os.getcwd().split('\\')[-1] != 'InsertDiffusion' and os.getcwd().split('/')[-1] != 'InsertDiffusion':
    os.chdir('..')

import utils
import importlib
importlib.reload(utils)
%load_ext autoreload
%autoreload 2

In [ ]:
from PIL import Image

fp = './test_images/for_eval/products/ref/'
product_filenames = os.listdir('./test_images/for_eval/products/ref/')

prompts = [x.split('.')[0].split('_')[0] for x in product_filenames]
product_descs = [x.split('.')[0].split('_')[1] for x in product_filenames]
product_images = [Image.open(os.path.join(fp, x)) for x in product_filenames]

In [ ]:
MASK_THRESHOLD = 210
INPAINTING_MODEL = 'stabilityai/stable-diffusion-2-inpainting'
INPAINTING_NEGATIVE_PROMPT = "wrong proportions, outline, white, collage, toy, silhouette, model, clay, above floor, flying, white background, duplicate, multiple, people, distortion, low quality, worst, ugly, fuzzy, blurry, cartoon, simple, art"
INPAINTING_GUIDANCE = 15
INPAINTING_STEPS = 75

IMG2IMG_MODEL = 'stabilityai/stable-diffusion-xl-refiner-1.0'
IMG2IMG_STRENGTH = 0.15  # changed from 0.2!
IMG2IMG_GUIDANCE = 7.5

def inpaint_product_image(img: Image, prompt: str, object_name: str, mask_threshold: int=225):
    try:
        extracted = utils.get_pasted_image(img, object_name, erosion_strength=3)
        repositioned = utils.paste_pipeline(extracted, 0.75, 0.5, 0.5, rescale=True)
    except:
        print('No erosion!')
        extracted = utils.get_pasted_image(img, object_name, erosion_strength=0)
        repositioned = utils.paste_pipeline(extracted, 0.75, 0.5, 0.5, rescale=True)
    mask = utils.get_mask_from_image(repositioned, mask_threshold)

    inpainted = utils.sd_inpainting(INPAINTING_MODEL, repositioned, mask, prompt, INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS)
    final = utils.sd_img2img(IMG2IMG_MODEL, inpainted, prompt, INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE)
    return final

In [ ]:
final_images = []
for i, img in enumerate(product_images):
    final = inpaint_product_image(img, prompts[i], product_descs[i], MASK_THRESHOLD)
    print(prompts[i])
    display(final)
    final_images.append(final)

In [ ]:
out_fp = './test_images/eval_results/ours_ablation/products/'
os.makedirs(out_fp, exist_ok=True)

for i, img in enumerate(final_images):
    fname = out_fp + prompts[i] + '.png'
    img.save(fname)

## Cars

In [ ]:
import os
if os.getcwd().split('\\')[-1] != 'InsertDiffusion' and os.getcwd().split('/')[-1] != 'InsertDiffusion':
    os.chdir('..')

import utils
import importlib
importlib.reload(utils)
%load_ext autoreload
%autoreload 2

In [ ]:
from PIL import Image

fp = './test_images/for_eval/cars/ref/'
car_filenames = sorted(os.listdir(fp))

prompts = [x.split('.')[0].split('_')[0] for x in car_filenames]
car_images = [Image.open(os.path.join(fp, x)) for x in car_filenames]

In [ ]:
MASK_THRESHOLD = 130
INPAINTING_MODEL = 'stabilityai/stable-diffusion-2-inpainting'
INPAINTING_NEGATIVE_PROMPT = "car, vehicle, cars, traffic, white background, floating, not on ground, flying, duplicate, multiple, people, antenna, distortion, low quality, worst, ugly, fuzzy, blurry, cartoon, simple, art"
INPAINTING_GUIDANCE = 15
INPAINTING_STEPS = 75

IMG2IMG_MODEL = 'stabilityai/stable-diffusion-xl-refiner-1.0'
IMG2IMG_STRENGTH = 0.2
IMG2IMG_GUIDANCE = 7.5

def inpaint_car_image(img: Image, prompt: str, mask_threshold: int=180):
    try:
        extracted = utils.get_pasted_image(img, 'car', erosion_strength=3)
        repositioned = utils.paste_pipeline(extracted, 0.75, 0.5, 0.5, rescale=True)
    except:
        print('No erosion!')
        extracted = utils.get_pasted_image(img, 'car', erosion_strength=0)
        repositioned = utils.paste_pipeline(extracted, 0.75, 0.5, 0.5, rescale=True)
    mask = utils.get_mask_from_image(repositioned, mask_threshold)

    inpainted = utils.sd_inpainting(INPAINTING_MODEL, repositioned, mask, prompt, INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS)
    final = utils.sd_img2img(IMG2IMG_MODEL, inpainted, prompt, INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE)
    return final

In [ ]:
final_images = []
for i, img in enumerate(car_images):
    final = inpaint_car_image(img, prompts[i], MASK_THRESHOLD)
    print(prompts[i])
    display(final)
    final_images.append(final)

In [ ]:
out_fp = './test_images/eval_results/ours_ablation/cars/'
os.makedirs(out_fp, exist_ok=True)

for i, img in enumerate(final_images):
    img.save(out_fp + prompts[i] + '.png')

# Composition

## Bikes

In [ ]:
import os
if os.getcwd().split('\\')[-1] != 'InsertDiffusion' and os.getcwd().split('/')[-1] != 'InsertDiffusion':
    os.chdir('..')

import utils
import importlib
importlib.reload(utils)
%load_ext autoreload
%autoreload 2

import pandas as pd
from PIL import Image

in_fp = './test_images/for_eval/bikes_composite'
bike_data = pd.read_csv(os.path.join(in_fp, 'bike_data.csv'))

bike_sketches = [Image.open(os.path.join(in_fp, 'sketches/' + x)) for x in sorted(os.listdir(os.path.join(in_fp, 'sketches')))]
backgrounds = [Image.open(bike_data.loc[i, 'background']) for i, _ in enumerate(bike_sketches)]

In [ ]:
COLORIZATION_MODEL = 'diffusers/stable-diffusion-xl-1.0-inpainting-0.1'
UPSCALING_MODEL = 'stabilityai/stable-diffusion-x4-upscaler'
COLORIZATION_NEGATIVE_PROMPT = "black and white, black frame, silhouette, motorbike, toy, clay, model, missing saddle, high saddle, details, detailed, greyscale, duplicate, multiple, detached, shadow, contact shadow, drop shadow, reflection, ground, unrealistic, bad, distorted, ugly, weird"
COLORIZATION_FILL_HOLES = True
COLORIZATION_DILATION = 8
COLORIZATION_STRENGTH = 0.91
COLORIZATION_GUIDANCE = 17
COLORIZATION_MASK_THRESHOLD = 170
COLORIZATION_STEPS = 30

INPAINTING_MASK_THRESHOLD = 200
INPAINTING_MODEL = 'stabilityai/stable-diffusion-2-inpainting'
INPAINTING_NEGATIVE_PROMPT = "wrong proportions, toy, black frame, motorbike, silhouette, model, clay, high saddle, large wheels, text, above floor, flying, changed bike color, white background, duplicate, multiple, people, basket, distortion, low quality, worst, ugly, fuzzy, blurry, cartoon, simple, art"
INPAINTING_GUIDANCE = 15
INPAINTING_STEPS = 75

IMG2IMG_MODEL = 'stabilityai/stable-diffusion-xl-refiner-1.0'
IMG2IMG_STRENGTH = 0.2
IMG2IMG_GUIDANCE = 7.5

In [ ]:
def paste_image(fg, bg):
    fg = fg.copy()
    bg = bg.copy()

    fg = fg.convert('RGBA')
    
    # replace white with tranparent
    pixdata = fg.load()
    bg_color = pixdata[0, 0]
    fuzziness = 20
    for y in range(fg.size[0]):
        for x in range(fg.size[1]):
            should_replace = True
            for idx in range(3):
                if abs(pixdata[x, y][idx] - bg_color[idx]) > fuzziness:
                    should_replace = False
            if should_replace:
                pixdata[x, y] = (0, 0, 0, 0)
    
    bg.paste(fg, (0, 0), fg)
    return bg

def do_insert_diffusion(fg: Image, bg: Image, positioning: dict, prompt: str, strength: float, img2img_strength: float):
    fg = utils.paste_pipeline(fg, positioning['scale'], positioning['fraction_down'], positioning['fraction_right'], False, rotation=positioning['rotation'])
    mask = utils.get_mask_from_image(fg, INPAINTING_MASK_THRESHOLD)

    if fg.width < bg.width:
        fraction = bg.width/img.width
        fg = fg.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
        mask = mask.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
    if fg.height < bg.height:
        fraction = bg.height/img.height
        fg = fg.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
        mask = mask.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)

    pasted_img = paste_image(fg, bg)
    inpainted = utils.sd_inpainting(INPAINTING_MODEL, pasted_img, mask, prompt, INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS, inpainting_strength=strength)
    final = utils.sd_img2img(IMG2IMG_MODEL, inpainted, prompt, INPAINTING_NEGATIVE_PROMPT, img2img_strength, IMG2IMG_GUIDANCE)
    return final

def colorization(image: Image, colorization_model: str, upscaling_model: str, colorization_prompt: str, colorization_negative_prompt: str, fill_holes: bool, dilation: int, strength: float, prompt_guidance: float):
    colorized = utils.sd_colorization(colorization_model, upscaling_model, image, colorization_prompt, negative_prompt=colorization_negative_prompt, fill_holes=fill_holes, dilation_iterations=dilation, colorization_strength=strength, prompt_guidance=prompt_guidance)
    return colorized

In [ ]:
STRENGTH = 0.38
IMG2IMG_STRENGTH = 0.2

result_imgs = []
for i, bike in enumerate(bike_sketches):
    colorized = colorization(bike, COLORIZATION_MODEL, UPSCALING_MODEL, bike_data.loc[i, 'col_prompt'], COLORIZATION_NEGATIVE_PROMPT, COLORIZATION_FILL_HOLES, COLORIZATION_DILATION, COLORIZATION_STRENGTH, COLORIZATION_GUIDANCE)
    curr = do_insert_diffusion(colorized, backgrounds[i], bike_data.iloc[i].to_dict(), bike_data.loc[i, 'inpainting_prompts'], STRENGTH, IMG2IMG_STRENGTH)
    print('Prompt: ', bike_data.loc[i, 'inpainting_prompts'])
    display(curr)
    result_imgs.append(curr)

In [ ]:
fp = './test_images/eval_results/ours_composite_ablation/bikes'
os.makedirs(fp, exist_ok=True)

for i, ri in enumerate(result_imgs):
    ri.save(os.path.join(fp, bike_data.loc[i, 'inpainting_prompts'] + '.png'))

## Cars

In [ ]:
import os
if os.getcwd().split('\\')[-1] != 'InsertDiffusion' and os.getcwd().split('/')[-1] != 'InsertDiffusion':
    os.chdir('..')

import utils
import importlib
importlib.reload(utils)
%load_ext autoreload
%autoreload 2

import pandas as pd
from PIL import Image

in_fp = './test_images/for_eval/cars_composite'
car_data = pd.read_csv(os.path.join(in_fp, 'car_data.csv'))

ref_files = os.listdir(os.path.join(in_fp, 'ref'))
ref_files.sort(key=lambda x: int(x.split('.')[0]))
bg_files = os.listdir(os.path.join(in_fp, 'backgrounds'))
bg_files.sort(key=lambda x: int(x.split('.')[0]))

car_images = [Image.open(os.path.join(in_fp, 'ref/' + x)) for x in ref_files]
backgrounds = [Image.open(os.path.join(in_fp, 'backgrounds/' + x)) for x in bg_files]

In [ ]:
COLORIZATION_MODEL = 'diffusers/stable-diffusion-xl-1.0-inpainting-0.1'
UPSCALING_MODEL = 'stabilityai/stable-diffusion-x4-upscaler'
COLORIZATION_NEGATIVE_PROMPT = "black and white, black frame, silhouette, motorbike, toy, clay, model, missing saddle, high saddle, details, detailed, greyscale, duplicate, multiple, detached, shadow, contact shadow, drop shadow, reflection, ground, unrealistic, bad, distorted, ugly, weird"
COLORIZATION_FILL_HOLES = True
COLORIZATION_DILATION = 8
COLORIZATION_STRENGTH = 0.91
COLORIZATION_GUIDANCE = 17
COLORIZATION_MASK_THRESHOLD = 170
COLORIZATION_STEPS = 30

INPAINTING_MASK_THRESHOLD = 200
INPAINTING_MODEL = 'stabilityai/stable-diffusion-2-inpainting'
INPAINTING_NEGATIVE_PROMPT = "wrong proportions, toy, black frame, motorbike, silhouette, model, clay, high saddle, large wheels, text, above floor, flying, changed bike color, white background, duplicate, multiple, people, basket, distortion, low quality, worst, ugly, fuzzy, blurry, cartoon, simple, art"
INPAINTING_GUIDANCE = 15
INPAINTING_STEPS = 75

IMG2IMG_MODEL = 'stabilityai/stable-diffusion-xl-refiner-1.0'
IMG2IMG_STRENGTH = 0.2
IMG2IMG_GUIDANCE = 7.5

def inpaint_car_image(fg: Image, bg: Image, prompt: str, positioning: dict, mask_threshold: int=180):
    fg = utils.extract_object(fg, bg, 'car')
    fg = utils.paste_pipeline(fg, positioning['scale'], positioning['fraction_down'], positioning['fraction_right'], rescale=False, rotation=positioning['rotation'])
    mask = utils.get_mask_from_image(fg, mask_threshold)

    if fg.width < bg.width:
        fraction = bg.width/img.width
        fg = fg.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
        mask = mask.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
    if fg.height < bg.height:
        fraction = bg.height/img.height
        fg = fg.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
        mask = mask.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
    
    pasted_img = utils.paste_image(fg, bg)

    inpainted = utils.sd_inpainting(INPAINTING_MODEL, pasted_img, mask, prompt, INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS, inpainting_strength=STRENGTH)
    final = utils.sd_img2img(IMG2IMG_MODEL, inpainted, prompt, INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE)
    return final

In [ ]:
STRENGTH = 0.38
IMG2IMG_STRENGTH = 0.2

final_images = []
for i, img in enumerate(car_images):
    final = inpaint_car_image(img, backgrounds[i], car_data.loc[i, 'prompt'], car_data.iloc[i].to_dict(), 230)
    print(car_data.loc[i, 'prompt'])
    display(final)
    final_images.append(final)

In [ ]:
fp = './test_images/eval_results/ours_composite_ablation/cars'
os.makedirs(fp, exist_ok=True)

for i, fi in enumerate(final_images):
    fi.save(os.path.join(fp, car_data.loc[i, 'prompt'] + '.png'))

## Products

In [ ]:
import os
if os.getcwd().split('\\')[-1] != 'InsertDiffusion' and os.getcwd().split('/')[-1] != 'InsertDiffusion':
    os.chdir('..')

import utils
import importlib
importlib.reload(utils)
%load_ext autoreload
%autoreload 2

import pandas as pd
from PIL import Image

in_fp = './test_images/for_eval/products_composite'
product_data = pd.read_csv(os.path.join(in_fp, 'product_data.csv'))

ref_files = os.listdir(os.path.join(in_fp, 'ref'))
ref_files.sort(key=lambda x: int(x.split('.')[0]))
bg_files = os.listdir(os.path.join(in_fp, 'backgrounds'))
bg_files.sort(key=lambda x: int(x.split('.')[0]))

product_images = [Image.open(os.path.join(in_fp, 'ref/' + x)) for x in ref_files]
backgrounds = [Image.open(os.path.join(in_fp, 'backgrounds/' + x)) for x in bg_files]

In [ ]:
MASK_THRESHOLD = 200
INPAINTING_MODEL = 'stabilityai/stable-diffusion-2-inpainting'
INPAINTING_NEGATIVE_PROMPT = "wrong proportions, outline, white, collage, toy, silhouette, model, clay, above floor, flying, white background, duplicate, multiple, people, distortion, low quality, worst, ugly, fuzzy, blurry, cartoon, simple, art"
INPAINTING_GUIDANCE = 15
INPAINTING_STEPS = 75

IMG2IMG_MODEL = 'stabilityai/stable-diffusion-xl-refiner-1.0'
IMG2IMG_STRENGTH = 0.15  # changed from 0.2!
IMG2IMG_GUIDANCE = 7.5

In [ ]:
def inpaint_product_image(fg: Image, bg: Image, prompt: str, object_desc: str, positioning: dict, mask_threshold: int=180):
    fg = utils.extract_object(fg, bg, object_desc)
    fg = utils.paste_pipeline(fg, positioning['scale'], positioning['fraction_down'], positioning['fraction_right'], rescale=False, rotation=positioning['rotation'])
    mask = utils.get_mask_from_image(fg, mask_threshold)

    if fg.width < bg.width:
        fraction = bg.width/img.width
        fg = fg.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
        mask = mask.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
    if fg.height < bg.height:
        fraction = bg.height/img.height
        fg = fg.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
        mask = mask.resize((int(fraction * fg.width), int(fraction * fg.height)), Image.Resampling.NEAREST)
    
    pasted_img = utils.paste_image(fg, bg)

    inpainted = utils.sd_inpainting(INPAINTING_MODEL, pasted_img, mask, prompt, INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS, inpainting_strength=STRENGTH)
    final = utils.sd_img2img(IMG2IMG_MODEL, inpainted, prompt, INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE)
    return final

In [ ]:
STRENGTH = 0.38
IMG2IMG_STRENGTH = 0.18

final_images = []

for i, product in enumerate(product_images):
    final = inpaint_product_image(product, backgrounds[i], product_data.loc[i, 'prompt'], product_data.loc[i, 'object_desc'], product_data.iloc[i].to_dict(), MASK_THRESHOLD)
    print(product_data.loc[i, 'prompt'])
    display(final)
    final_images.append(final)

In [ ]:
fp = './test_images/eval_results/ours_composite_ablation/products'
os.makedirs(fp, exist_ok=True)

for i, fi in enumerate(final_images):
    fi.save(os.path.join(fp, product_data.loc[i, 'prompt'] + '.png'))